# Table S05: Comparison of the mean standard deviation of normalized

animal lengths across toxicant groups

In [ ]:
library(tidyverse)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'flextable'

The following object is masked from 'package:purrr':

    compose

In [ ]:
# Source the output functions
source(file.path("bin/outs.R"))



Attaching package: 'data.table'

The following objects are masked from 'package:lubridate':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year

The following objects are masked from 'package:dplyr':

    between, first, last

The following object is masked from 'package:purrr':

    transpose


Attaching package: 'kableExtra'

The following objects are masked from 'package:flextable':

    as_image, footnote

The following object is masked from 'package:dplyr':

    group_rows

here() starts at /Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript

$strain_table
$strain_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.html"

$strain_table$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.tsv.zip"

$strain_table$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.docx"

$strain_table$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S1/table_S1.csv"


$tox_table_ft
$tox_table_ft$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.html"

$tox_table_ft$tsv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.tsv.zip"

$tox_table_ft$docx
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.docx"

$tox_table_ft$csv
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S2/table_S2.csv"


$trait_table
$trait_table$html
[1] "/Users/ryanmckeown/Desktop/ToxinGWAS_Manuscript/tables/table_S3/table_S3.html"

$trait_table$tsv
[1] "

# Inputs

In [ ]:
# Load required data
drug_means <- readRDS(file.path("data/processed/phenotypes/drug_means.rds"))


# Main

In [ ]:

# Calculate summary statistics for both Use and MoA groups
use_group_summary <- drug_means %>%
  dplyr::group_by(group = big_class) %>%
  dplyr::summarise(
    group_type = "Use",
    avg_sd = mean(sd_NormLength),
    sd_avg_sd = sd(sd_NormLength)
  )

moa_group_summary <- drug_means %>%
  dplyr::group_by(group = moa_class) %>%
  dplyr::summarise(
    group_type = "MoA",
    avg_sd = mean(sd_NormLength),
    sd_avg_sd = sd(sd_NormLength)
  )

# Combine the summaries
group_summary <- dplyr::bind_rows(use_group_summary, moa_group_summary) %>%
  dplyr::arrange(group_type, desc(avg_sd))


``` r
group_summary
```

    # A tibble: 8 × 4
      group            group_type avg_sd sd_avg_sd
      <chr>            <chr>       <dbl>     <dbl>
    1 Redox disruption MoA          39.1     14.4 
    2 AChE inhibition  MoA          38.5     17.1 
    3 Other|unknown    MoA          32.8     15.9 
    4 Insecticide      Use          43.9     16.2 
    5 Metal            Use          38.2     19.1 
    6 Herbicide        Use          33.5      5.42
    7 Flame Retardant  Use          31.5      8.97
    8 Fungicide        Use          30.2     12.2

In [ ]:
# Create grouped data
grouped_data <- as_grouped_data(
  x = group_summary,
  groups = "group_type"
)

# Create flextable from grouped data
group_pheno_ft <- as_flextable(
  grouped_data,
  col_keys = c(
    "group",
    "avg_sd",
    "sd_avg_sd"
  ),
  hide_grouplabel = TRUE
) %>%
  flextable::set_header_labels(
    values = list(
      "group" = "Group",
      "avg_sd" = "Average SD"
    )
  ) %>%
  # Combine avg_sd and sd_avg_sd using mk_par
  flextable::compose(
    j = "avg_sd",
    # but not for the group rows
    i = ~ !is.na(group),
    value = as_paragraph(
      sprintf("%.3f", avg_sd),
      " ± ",
      sprintf("%.3f", sd_avg_sd)
    )
  ) %>%
  # Delete the sd_avg_sd column since it's now combined
  flextable::delete_columns("sd_avg_sd") %>%
  # make only the groups and header bold
  flextable::bold(
    j = 1,
    i = ~ is.na(group),
    bold = TRUE,
    part = "body"
  ) %>%
  flextable::bold(part = "header") %>%
  flextable::line_spacing(space = 1, part = "all") %>%
  flextable::align(j = 1, align = "center") %>%
  flextable::align(j = 2, align = "center") %>%
  flextable::hline(
    i = ~ is.na(group),
    part = "body"
  ) %>%
  flextable::set_table_properties(
    layout = "autofit",
    width = 1,
    opts_word = list(
      split = FALSE,
      keep_with_next = TRUE
    )
  )


# Output

In [ ]:
# Save the table using standardized functions
save_supp_table_csv(group_summary, "group_summary")
save_supp_table_flextable(group_pheno_ft, "group_summary")
